# Exercise 04 : Enrichment and transformations

In [1]:
import pandas as pd
import numpy as np

## Read the JSON file that you saved in ex02

- one of the columns has the float type, so let us define the format of it in pandas using pd.options.display.float_format: floats should be displayed with two decimals
- there are values missing from the Model, do not do anything with them

In [2]:
df = pd.read_json('../ex02/auto.json')
pd.options.display.float_format = '{:.2f}'.format

## Enrich the dataframe using a sample from that dataframe

- create a sample with 200 new observations with random_state = 21
  - the sample should not have new combinations of the car number, make and model, so the whole dataset will be consistent in these terms
  - there are no restrictions on the refund and fines, you can take any value from these columns at random and use it towards any car number
- concatenate the sample with the initial dataframe to a new dataframe concat_rows

In [3]:
np.random.seed(21)
sample = df.sample(n=200, replace=True)

concat_rows = pd.concat([df, sample])

## Enrich the dataframe concat_rows by a new column with the data generated

- create a series with the name Year using random integers from 1980 to 2019
- use np.random.seed(21) before generating the years
- concatenate the series with the dataframe and name it fines

In [4]:
np.random.seed(21)
concat_rows = concat_rows.reset_index(drop=True)
years = pd.Series(np.random.randint(1980, 2020, size=len(concat_rows)), name='Year')
fines = pd.concat([concat_rows, years], axis=1)

## Enrich the dataframe with the data from another dataframe

- create a new dataframe with the car numbers and their owners
  - get the most popular surnames(you can find the file surname.json in the attachments) in the US
  - create a new series with the surnames (they should not have special characters like commas, brackets, etc.) from the data you gathered, the count should be equal to the number of unique car numbers using the sample (use random_state = 21)
  - create the dataframe owners with 2 columns: CarNumber and SURNAME

- append 5 more observations to the fines dataframe (come up with your own ideas of CarNumber, etc.)
- delete the dataframe last 20 observations from the owners and add 3 new observations (they are not the same as those you add to the fines dataframe)
- join both dataframes:
  - the new dataframe should have only the car numbers that exist in both dataframes
  - the new dataframe should have all the car numbers that exist in both dataframes
  - the new dataframe should have only the car numbers from the fines dataframe
  - the new dataframe should have only the car numbers from the owners dataframe

In [5]:
np.random.seed(21)

# Create a dataframe with car numbers and owners
surnames = pd.read_json('surnames.json').squeeze()
owners_car_numbers = fines['CarNumber'].unique()
owners_surnames = np.random.choice(surnames, size=len(owners_car_numbers))
owners = pd.DataFrame({'CarNumber': owners_car_numbers, 'SURNAME': owners_surnames})

In [6]:
# Append 5 more observations to the fines dataframe
additional_observations = pd.DataFrame({
    'CarNumber': ['Z999ZZ99RUS', 'X888XX88RUS', 'Y777YY77RUS', 'W666WW66RUS', 'V555VV55RUS'],
    'Refund': [1.0, 2.0, 1.0, 3.0, 2.0],
    'Fines': [2500.0, 3500.0, 2000.0, 4000.0, 3000.0],
    'Make': ['Toyota', 'Ford', 'Nissan', 'Chevrolet', 'Honda'],
    'Model': ['Camry', 'Focus', 'Altima', 'Cruze', 'Accord']
})
fines = pd.concat([fines, additional_observations])

In [7]:
# Delete the last 20 observations from the owners and add 3 new observations
owners = owners.iloc[:-20]
additional_owners = pd.DataFrame({
    'CarNumber': ['A111AA11RUS', 'B222BB22RUS', 'C333CC33RUS'],
    'SURNAME': ['Smith', 'Johnson', 'Williams']
})
owners = pd.concat([owners, additional_owners])

In [8]:
# Join dataframes
# Only the car numbers that exist in both dataframes
common_car_numbers_inner = pd.merge(fines, owners, on='CarNumber', how='inner')

# All the car numbers that exist in both dataframes
common_car_numbers_outer = pd.merge(fines, owners, on='CarNumber', how='outer')

# Only the car numbers from the fines dataframe
car_numbers_from_fines = fines['CarNumber']
fines_car_numbers = fines[fines['CarNumber'].isin(owners['CarNumber'])]

# Only the car numbers from the owners dataframe
car_numbers_from_owners = owners['CarNumber']
owners_car_numbers = owners[owners['CarNumber'].isin(fines['CarNumber'])]

## Create a pivot table from the fines dataframe

In [9]:
pivot_table = fines.pivot_table(index=['Make', 'Model'], columns='Year', values='Fines', aggfunc='sum')
pivot_table

Year                1980.00   1981.00   1982.00  1983.00   1984.00   1985.00   
Make       Model                                                               
Ford       Focus   62394.59 395589.17 140383.76 63100.00 111294.59 189583.76  \
           Mondeo       NaN       NaN       NaN      NaN       NaN       NaN   
Skoda      Octavia 12494.59       NaN   6900.00 11594.59   1200.00  10294.59   
Toyota     Camry   18500.00   8594.59       NaN  7200.00       NaN       NaN   
           Corolla      NaN       NaN   2000.00      NaN       NaN       NaN   
Volkswagen Golf    30900.00       NaN       NaN  8594.59    300.00  24000.00   
           Jetta        NaN       NaN       NaN      NaN       NaN       NaN   
           Passat       NaN   4600.00       NaN  3200.00  10000.00   5000.00   
           Touareg      NaN       NaN       NaN      NaN       NaN   5800.00   

Year                1986.00   1987.00  1988.00   1989.00  ...   2010.00   
Make       Model                                          ...             
Ford       Focus   88994.59 121900.00 95989.17 115500.00  ... 120183.76  \
           Mondeo       NaN       NaN      NaN   8600.00  ...       NaN   
Skoda      Octavia   600.00  26700.00      NaN  91400.00  ...   3100.00   
Toyota     Camry        NaN       NaN      NaN  22400.00  ...       NaN   
           Corolla 16000.00   8000.00      NaN   4000.00  ...  24000.00   
Volkswagen Golf         NaN   9300.00      NaN   8100.00  ...       NaN   
           Jetta        NaN       NaN      NaN       NaN  ...       NaN   
           Passat  15000.00  12300.00      NaN       NaN  ...   2800.00   
           Touareg      NaN       NaN      NaN       NaN  ...   6300.00   

Year                2011.00   2012.00   2013.00   2014.00   2015.00  2016.00   
Make       Model                                                               
Ford       Focus   86689.17 112700.00 145494.59 158894.59 203694.59 72594.59  \
           Mondeo       NaN  34400.00       NaN       NaN       NaN 46200.00   
Skoda      Octavia   500.00    500.00  13794.59   1900.00  46394.59   300.00   
Toyota     Camry        NaN  16094.59       NaN       NaN       NaN 13000.00   
           Corolla  8594.59       NaN       NaN       NaN       NaN      NaN   
Volkswagen Golf      600.00       NaN   9300.00       NaN   2300.00      NaN   
           Jetta        NaN       NaN       NaN       NaN       NaN      NaN   
           Passat   3000.00   2000.00    300.00       NaN   9194.59  2100.00   
           Touareg      NaN       NaN       NaN   1300.00    500.00      NaN   

Year                 2017.00   2018.00   2019.00  
Make       Model                                  
Ford       Focus   272200.00 285194.59 101100.00  
           Mondeo        NaN       NaN       NaN  
Skoda      Octavia       NaN 156200.00   9500.00  
Toyota     Camry     1000.00  13000.00  18100.00  
           Corolla   9600.00       NaN  16000.00  
Volkswagen Golf          NaN       NaN       NaN  
           Jetta         NaN       NaN       NaN  
           Passat        NaN       NaN       NaN  
           Touareg       NaN       NaN       NaN  

[9 rows x 40 columns]

## Save both the fines and owners dataframes to CSV files without an index

In [10]:
fines.to_csv('fines.csv', index=False)
owners.to_csv('owners.csv', index=False)